# API Google Sheets
Для работы с Google Sheets сначала необходимо пройти процедуру авторизации с помощью Google Developers Console.

Она аналогична той, что мы делали в прошлом модуле для API Google Analytics.

В этот раз все должно пройти проще. Итак, заходим в консоль разработчика https://console.developers.google.com. В ней должен остаться доступ к Google Analytics Reporting API с прошлого модуля (но в данном модуле он нам не пригодится). Нажимаем на "Включить API и сервисы":

# Запрос токенов
Создайте jupyter notebook в той же папке, куда вы сохранили файл с индентификаторами client_secret_grants.json.

Начинаем как обычно с импорта необходимых библиотек (те же, что мы использовали для авторизации Google Analytics):

In [1]:
import httplib2
import argparse
from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

import pandas as pd
import numpy as np

Задаем основные параметры приложения, от имени которого будем делать запросы к Google Sheets API:

In [2]:
# разрешения на просмотр и редактирование

SCOPES = 'https://www.googleapis.com/auth/spreadsheets'

# название файла с индентификаторами

CLIENT_SECRET_FILE = './module14_files/client_secret_grants.json'

# Название приложения для нашего онлайн-отчета (можно использовать любое)

APPLICATION_NAME = 'Google Sheets API Report'

# название файла, в который после процедуры авторизации будут записаны токены (сейчас его еще не должно быть в папке)

# для Google Analytics этот файл назывался 'analytics.dat', но в разработчики Google Sheets API выбрали другое название

# чтобы вам при необходимости было проще ориентироваться в документации Google API мы сохранили такую же систему обозначений

credential_path = 'sheets.googleapis.com-report.json'

Проходим процесс авторизации для работы с Google Sheets в первый раз:

В открывшемся окне выбираем нужный аккаунт (если у вас их несколько)
Даем вашему приложению разрешение на "Просмотр и изменение электронных таблиц на Google Диске"
Процесс должен завершиться со служебным сообщением "The authentication flow has completed"

In [3]:
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])

flags = parser.parse_args([])

if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    
    if flags:
        credentials = tools.run_flow(flow, store, flags)
        
    print('Storing credentials to ' + credential_path)

В вашей папке должен появиться файл "sheets.googleapis.com-report.json", в котором содержится токен access_token для запросов к API Google Sheets и refresh_token для его обновления.

Формируем объект service, с помощью которого мы будем читать и записывать данные Google Sheets:

In [4]:
http = credentials.authorize(httplib2.Http())

discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')

service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)

# Чтение данных из Google Sheets
На данном шаге мы научимся считывать данные из Google-таблиц.

Этот алгоритм может сильно упростить вам многие задачи обработки данных, т. к. вы и ваши коллеги можете обновлять необходимые словари, зависимости и другие таблицы онлайн. К тому же в Google Sheets можно распределять доступы на чтение и редактирование таблиц.

Давайте разберем это на примере ведения отчетности маркетинговых кампаний.

Допустим, на ваш сайт приходит трафик от нескольких партнеров (назовем их СМИ2, РБК и Infox). У каждого партнера в ссылках на ваш сайт имеется свой набор UTM-меток.

Справочная информация: под UTM-меткой подразумевается принятая в маркетинге система обозначений параметров ссылок, которые автоматически распознаются многими системами веб-аналитики как название источника и канала. Пример ссылки с UTM-метками:

https://my-site.ru/?utm_source=infox&utm_medium=cpc

Вам необходимо составить отчет по количеству визитов, продаж и прочим метрикам в разбивке по партнерам СМИ2, РБК и Infox.

Проблема в том, что названия UTM-меток могут меняться (обычно так и случается в реальной жизни). Конечно, можно договориться о том, что вам при любом обновлении будут присылать обновленный список названий UTM-меток. Скорее всего работать это не будет.

Более простым вариантом является ведение онлайн-таблицы, в которой будут отмечаться эти изменения.

Обычно такие таблицы в том или ином виде ведутся (иногда, правда, в Excel). Перенести их в Google Sheets обычно не составляет организационных проблем.

В итоге если вам составят обновляемую таблицу с UTM-метками, то вы (и ваш скрипт) всегда сможете узнать какой набор UTM-меток соответствует какому партнеру:
<img src='./module14_files/m9_b2_labels_step1.png'>

#### Шаг 1.

Создайте в своем аккаунте Google Sheets (https://docs.google.com/spreadsheets) новую таблицу (например, с названием "Словарь меток"). Скопируйте из файла Labels.xlsx таблицу словаря меток в созданную таблицу:

Файл labels.xlsx

#### Шаг 2. 

Для того, чтобы автоматически прочитать ее содержимое, нам понадобятся два параметра:

1. ID книги Google Sheets. Для ее получения скопируйте адрес страницы, на которой расположена ваша таблица словаря меток.

URL должен иметь вид https://docs.google.com/spreadsheets/d/4P57_g3hfhrihuqf-2f49t7ghwifwtyr-4UB45/edit#gid=12345

Нужным ID книги является идентификатор между "d/" и "/edit". Т. е. в данном примере это "4P57_g3hfhrihuqf-2f49t7ghwifwtyr-4UB45". Запишем его в переменную spreadsheetId:

In [5]:
spreadSheetId = '1XlUS0O1TILVXPfMPrWt4q36_32ofxAGyv-8aw9AvvGk'

2. Для чтения данных нам также необходимо указать название листа (если их несколько) и диапазон.

Например, в данном модуле у нас будет 2 листа: текущий можно назвать "partners dict", а также мы сделаем онлайн-отчет на листе "daily_metrics".

Сама таблица помещается в диапазон A1:C5. Поскольку этот список в будущем может вырасти, то возьмем с запасом до строки 10:

In [6]:
rangeName = 'partners_dict!A1:C10'

Все готово, делаем запрос к API. Результат записываем в переменную values:

In [7]:
result = (service.spreadsheets().values()
          .get(spreadsheetId=spreadSheetId, 
               range=rangeName)
          .execute())

values = result.get('values', [])

In [8]:
print(values)

[['Партнер', 'utm_source', 'utm_medium'], ['СМИ2', 'smi2', 'cpm'], ['РБК', 'rbc', 'cpc'], ['Infox', 'infox', 'cpc'], ['Infox', 'infox-action', 'cpa']]


Запрос автоматически убрал из ответа пустые строки. Теперь лист values можно использовать как словарь для вашей отчетности по партнерам.

# Упражнение (подсказки есть ниже)
(1 возможный балл)
В файле metrika_data.txt (ссылка на него ниже в рекомендациях) имеется отчет Яндекс.Метрики по UTM-меткам. Объедините его с листом values и посчитайте сумму продаж (столбец Sales) для партнера Infox.

Какая сумма продаж получается у партнера Infox?

In [9]:
metrika_data = pd.read_csv('./module14_files/metrika_data.txt', sep='\t')

partners_data = pd.DataFrame(values[1:], columns=values[0])

In [10]:
metrika_data

,utm_source,utm_medium,sales
0,infox,cpc,1534
1,smi2,cpm,1495
2,infox-action,cpa,735
3,rbc,cpc,59


In [11]:
partners_data

,Партнер,utm_source,utm_medium
0,СМИ2,smi2,cpm
1,РБК,rbc,cpc
2,Infox,infox,cpc
3,Infox,infox-action,cpa


In [12]:
partners_data.merge(metrika_data, on='utm_source').groupby('Партнер')['sales'].sum()

Партнер
Infox    2269
РБК        59
СМИ2     1495
Name: sales, dtype: int64

# Подготовка отчета
Теперь у нас все готово, чтобы сделать полноценную отчетность в Google Sheets, данные в которой будут обновляться полностью автоматически. Давайте в этом шаге рассмотрим следующую задачу: необходимо составить ежедневный мониторинг основных метрик сайта из Яндекс.Метрики за последние 14 дней. Т. е. получить в Google Sheets следующий отчет:
<img src='./module14_files/m9_b2_write_step1.png'>
Яндекс.Метрика взята для примера. На месте этого мониторинга может быть любой отчет или результат ваших вычислений.


Сначала в таблице Googlesheets из прошлых шагов "Словарь меток" добавим новый лист daily_metrics и сразу наметим столбцы будущего отчета:
<img src='./module14_files/m9_b2_write_step2.png'>

Подготовимся к импорту данных из Яндекс.Метрики: в файл config.yaml пишем токен для запросов к API. В файле params.yaml в строку с spreadsheet_id необходимо вставить ID вашей книги. В файле report_params.yaml записываются параметры отчетов, которые мы хотим визуализировать:

Файл report_params.yaml

- dau - служебное название отчета. Если вы захотите добавить еще один отчет, то сделайте копию этого названия со всем параметрами под ним
- dimensions и metrics - измерения и метрики, которые будем отправлять в запросе к API Яндекс.Метрики
- sort - метрика, по которой будем сортировать отчет. В нашем случае это дата отчета
- days_ago - за какой период выгружаем отчет. В нашем варианте - за последние 14 дней
- spreadsheet_id - ID книги в Google Sheets, где расположен отчет (подставьте ID своей книги)
- range_name - название листа отчета (daily_metrics), диапазон столбцов, в которых будет располагаться выгрузка (с A по E)

Далее импортируем те же методы и функции, которые использовали при работе с API Яндекс.Метрики:

In [13]:
from yaml import load
from datetime import datetime, timedelta
import requests
from pprint import pprint

In [14]:
# Импортируем токен для запросов к API:

with open('../module12/module12_files/yandex_config.yaml', 'r', encoding='utf-8') as f:
    token = load(f)['access_token']

# Импортируем параметры работы скрипта из report_params.yaml:

with open('./module14_files/report_params.yaml', 'r') as f:
    params = load(f)

API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = (datetime.now() - timedelta(days = params['reports']['dau']['days_ago'])).strftime('%Y-%m-%d')
endDate = (datetime.now() - timedelta(days = 1)).strftime('%Y-%m-%d')
dimensions = params['reports']['dau']['dimensions']
metrics = params['reports']['dau']['metrics']
sortby = params['reports']['dau']['sort']
counter = params['metrika counter']

# Важен порядок передачи параметров!!!

In [15]:
yandex_api_params = {
    'date1': startDate,
    'date2': endDate,
    'dimensions': dimensions,
    'metrics': metrics,
    'id': counter,
    'sort': sortby,
    'oauth_token': token,
    'accuracy': 1
}

In [16]:
# Используем функции для запроса и обработки ответа API из прошлых модулей:

def make_request(url, **kwargs):
    
    """Запрос к API Яндекс.Метрики. Возвращает JSON ответа"""

    return requests.get(url, params=kwargs).json()

def json_handler(data):

    """Преобразование ответа из JSON в список"""

    report = []
    
    if 'data' in data:
        for line in data['data']:
            dimensions_list = [x['name'] for x in line['dimensions']]
            metrics_list = line['metrics']
            report.append( dimensions_list + metrics_list )  
            
        return report
    
    print('Error: {}'.format(data['message']))
    return data

In [18]:
# Делаем запрос к Яндекс.Метрике:

response = make_request(API_URL, **yandex_api_params)
data = json_handler(response)

In [19]:
data

[['2018-12-06', 152.0, 141.0, 213.0, 15.13157895],
 ['2018-12-07', 181.0, 159.0, 326.0, 17.12707182],
 ['2018-12-08', 157.0, 142.0, 256.0, 14.64968153],
 ['2018-12-09', 137.0, 129.0, 207.0, 13.13868613],
 ['2018-12-10', 158.0, 142.0, 243.0, 12.65822785],
 ['2018-12-11', 144.0, 131.0, 232.0, 22.22222222],
 ['2018-12-12', 136.0, 127.0, 191.0, 10.29411765],
 ['2018-12-13', 186.0, 171.0, 271.0, 23.65591398],
 ['2018-12-14', 150.0, 132.0, 225.0, 18.0],
 ['2018-12-15', 166.0, 155.0, 260.0, 12.65060241],
 ['2018-12-16', 143.0, 139.0, 201.0, 16.08391608],
 ['2018-12-17', 158.0, 150.0, 239.0, 14.55696203],
 ['2018-12-18', 151.0, 136.0, 261.0, 11.9205298],
 ['2018-12-19', 123.0, 118.0, 169.0, 13.82113821]]

# Запись данных в Google Sheets
Данные получили, осталось отправить их в Google Sheets. Сначала сформируем название диапазона (daily_metrics!A2:E15), в который отправим данные Метрики:

In [24]:
range_data = params['reports']['dau']['range_name']
spreadsheet_id = params['reports']['dau']['spreadsheet_id']

range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)

# Проверяем верен ли диапазон:

print(range_name)

daily_metrics!A2:E15


In [21]:
# Для отправки листа data в Google Sheets его необходимо преобразовать в словарь:

body = {'values': data}

pprint(body)

{'values': [['2018-12-06', 152.0, 141.0, 213.0, 15.13157895],
            ['2018-12-07', 181.0, 159.0, 326.0, 17.12707182],
            ['2018-12-08', 157.0, 142.0, 256.0, 14.64968153],
            ['2018-12-09', 137.0, 129.0, 207.0, 13.13868613],
            ['2018-12-10', 158.0, 142.0, 243.0, 12.65822785],
            ['2018-12-11', 144.0, 131.0, 232.0, 22.22222222],
            ['2018-12-12', 136.0, 127.0, 191.0, 10.29411765],
            ['2018-12-13', 186.0, 171.0, 271.0, 23.65591398],
            ['2018-12-14', 150.0, 132.0, 225.0, 18.0],
            ['2018-12-15', 166.0, 155.0, 260.0, 12.65060241],
            ['2018-12-16', 143.0, 139.0, 201.0, 16.08391608],
            ['2018-12-17', 158.0, 150.0, 239.0, 14.55696203],
            ['2018-12-18', 151.0, 136.0, 261.0, 11.9205298],
            ['2018-12-19', 123.0, 118.0, 169.0, 13.82113821]]}


Наконец, отправляем эти данные в Google Sheets. Опция valueInputOption дает команду API на то, что значения надо вставить так как если бы пользователь ввел их вручную в веб-интерфейсе. Подробнее: https://developers.google.com/sheets/api/reference/rest/v4/ValueInputOption

In [23]:
params

{'metrika counter': 21075004,
 'reports': {'dau': {'dimensions': 'ym:s:date',
   'metrics': 'ym:s:visits,ym:s:users,ym:s:pageviews,ym:s:bounceRate',
   'sort': 'ym:s:date',
   'days_ago': 14,
   'spreadsheet_id': '1XlUS0O1TILVXPfMPrWt4q36_32ofxAGyv-8aw9AvvGk',
   'range_name': {'list': 'daily_metrics', 'start': 'A', 'end': 'E'}}}}

In [25]:
(service.spreadsheets().values()
 .update(spreadsheetId=spreadsheet_id, 
         range=range_name, 
         valueInputOption='USER_ENTERED', 
         body=body)
 .execute())

{'spreadsheetId': '1XlUS0O1TILVXPfMPrWt4q36_32ofxAGyv-8aw9AvvGk',
 'updatedRange': 'daily_metrics!A2:E15',
 'updatedRows': 14,
 'updatedColumns': 5,
 'updatedCells': 70}

Теперь проверим книгу в Google Sheets. У вас должен получиться подобный результат:

<img src='./module14_files/m9_b2_write_step5.png'>

# Упражнение
(1 возможный балл)
Получите аналогичную таблицу в своей книге Google Sheets за фиксированные даты с 25 февраля по 10 марта 2018 года.

Чему равна сумма визитов за этот период?

In [28]:
yandex_api_params['date1'] = '2018-02-25'
yandex_api_params['date2'] = '2018-03-10'

In [29]:
response = make_request(API_URL, **yandex_api_params)
data = json_handler(response)

In [33]:
range_data = params['reports']['dau']['range_name']
spreadsheet_id = params['reports']['dau']['spreadsheet_id']

range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)

# Проверяем верен ли диапазон:
print(range_name)

# Для отправки листа data в Google Sheets его необходимо преобразовать в словарь:
body = {'values': data}

pprint(body)

daily_metrics!A2:E15
{'values': [['2018-02-25', 202.0, 193.0, 386.0, 15.34653465],
            ['2018-02-26', 279.0, 264.0, 440.0, 13.62007168],
            ['2018-02-27', 280.0, 258.0, 481.0, 18.21428571],
            ['2018-02-28', 294.0, 278.0, 399.0, 15.98639456],
            ['2018-03-01', 293.0, 272.0, 430.0, 15.35836177],
            ['2018-03-02', 317.0, 284.0, 441.0, 12.30283912],
            ['2018-03-03', 342.0, 292.0, 551.0, 15.78947368],
            ['2018-03-04', 265.0, 233.0, 429.0, 19.62264151],
            ['2018-03-05', 282.0, 257.0, 427.0, 17.0212766],
            ['2018-03-06', 258.0, 229.0, 389.0, 17.82945736],
            ['2018-03-07', 203.0, 185.0, 318.0, 12.31527094],
            ['2018-03-08', 198.0, 168.0, 382.0, 11.61616162],
            ['2018-03-09', 258.0, 232.0, 514.0, 18.21705426],
            ['2018-03-10', 221.0, 198.0, 429.0, 20.36199095]]}


In [34]:
(service.spreadsheets().values()
 .update(spreadsheetId=spreadsheet_id, 
         range=range_name, 
         valueInputOption='USER_ENTERED', 
         body=body)
 .execute())

{'spreadsheetId': '1XlUS0O1TILVXPfMPrWt4q36_32ofxAGyv-8aw9AvvGk',
 'updatedRange': 'daily_metrics!A2:E15',
 'updatedRows': 14,
 'updatedColumns': 5,
 'updatedCells': 70}